In [2]:
import requests
import json
from urllib.parse import quote
import httpx
import jmespath

In [5]:
def scrape_post(post_url):
    """Scrape single Instagram post data"""
    
    shortcode = post_url.split('/p/')[-1].split('/')[0] if 'http' in post_url else post_url
    print(f"Scraping: {shortcode}")
    
    graph_api = "https://www.instagram.com/graphql/query/?query_hash=b3055c01b4b222b8a47dc12b090e4e64&variables="
    variables = {
        "shortcode": shortcode,  # post shortcode (from URL)
        "child_comment_count": 20,  
        "fetch_comment_count": 100,
        "parent_comment_count": 24,
        "has_threaded_comments": True
        }
    headers = {
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'x-asbd-id': '129477',
        'x-ig-app-id': '936619743392459',
        'cookie': '''csrftoken=cpBvgDRGLIQ4j3RgScGvH24w9R6cQffX; ds_user_id=18519728814; ps_n=1; ps_l=1; mid=Zl0FuQALAAHWLneH1O51KnPmZbQO; ig_did=8F185B9B-0018-4D2C-9C8B-5861E8826960; dpr=0.8640000224113464; datr=fid7ZoVfi9JAdPRsF4AUt5H9; shbid="19508\05418519728814\0541751227322:01f71e1583bc74deb0cf35aadeef14cbf22516af31cc981c85d6a595394c0fc9c9797cbb"; shbts="1719691322\05418519728814\0541751227322:01f797a9ebf1355c2fe47a789c2fca5f528433508e5c351bbb67dc69bcb3920b918ea3d3"; sessionid=18519728814%3A8CptQEBMl6lstb%3A29%3AAYfPHfaU0TD5O2F2otwE2BteTZDIh3lS8N5qbid1QQ; rur="RVA\05418519728814\0541751409798:01f7914db0d124d7da5a7af3e8dbeca4c2fc69e72380836ba0862459726e5ae2d6f8640f"; wd=1582x268'''
        }
    
    quoted_vars = quote(json.dumps(variables))
    final_url = graph_api + quoted_vars
    response = httpx.get(final_url, headers=headers)
    assert response.status_code == 200
    return response.json()['data']['shortcode_media']

In [12]:
post = "https://www.instagram.com/p/C8djxnMiM9P/?hl=en&img_index=1"
data = scrape_post(post)

Scraping: C8djxnMiM9P


In [14]:
data

{'__typename': 'GraphSidecar',
 'id': '3395027035075694415',
 'shortcode': 'C8djxnMiM9P',
 'dimensions': {'height': 1348, 'width': 1080},
 'gating_info': None,
 'fact_check_overall_rating': None,
 'fact_check_information': None,
 'sensitivity_friction_info': None,
 'sharing_friction_info': {'should_have_sharing_friction': False,
  'bloks_app_url': None},
 'media_overlay_info': None,
 'media_preview': None,
 'display_url': 'https://instagram.fcai20-1.fna.fbcdn.net/v/t51.29350-15/448392955_861069055833085_7242564436392353641_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fcai20-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=aWTipho2DToQ7kNvgE-UdGn&edm=AP_V10EBAAAA&ccb=7-5&oh=00_AYAzvwWkcNqh_5AckgMYP4HxBnqVtyd7u9S0Y2DZ-DFr2Q&oe=668B126A&_nc_sid=2999b8',
 'display_resources': [{'src': 'https://instagram.fcai20-1.fna.fbcdn.net/v/t51.29350-15/448392955_861069055833085_7242564436392353641_n.jpg?stp=dst-jpg_e35_p640x640_sh0.08&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDE3OTguc2RyLmYyOTM1MCJ9&_nc_

In [30]:
result = jmespath.search('''
    {
    id: id,
    shortcode: shortcode,
    dimentions: dimentions,
    url: shortcode,
    src_attached: edge_sidecar_to_children.edges[].node.display_url,
    has_audio: has_audio,
    video_url: video_url,
    views: video_view_count,
    plays: video_play_count,
    likes: edge_media_preview_like.count,
    location: location.name,
    taken_at: taken_at_timestamp,
    related: edge_web_media_to_related_media.edges[].node.shortcode,
    type: product_type,
    video_duration: video_duration,
    music: clips_music_attribution_info,
    is_video: is_video,
    tagged_users: edge_media_to_tagged_user.edges[].node.user.username,
    captions: edge_media_to_caption.edges[].node.text,
    related_profiles: edge_related_profiles.edges[].node.username,
    comments_count: edge_media_to_parent_comment.count,
    comments_disabled: comments_disabled,
    comments_next_page: edge_media_to_parent_comment.page_info.end_cursor,
    comments: edge_media_to_parent_comment.edges[].node.{
        id: id,
        text: text,
        created_at: created_at,
        viewer_has_liked: viewer_has_liked,
        likes: edge_liked_by.count
        owner: owner.{
            username: username
            is_verified: is_verified,
            profile_pic: profile_pic_url
        }

    }
    }
''', data)

In [29]:
result['url'] = f"https://instagram.com/p/result['url']
result['url']
https://www.instagram.com/p/C86aabgxXNC/?hl=en

'https://instagram.com/C8djxnMiM9P'

In [11]:
import json
from typing import Dict
from urllib.parse import quote

import httpx

INSTAGRAM_APP_ID = "936619743392459"  # this is the public app id for instagram.com


def scrape_post(url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode
    print(f"scraping instagram post: {shortcode}")
    
    variables = {
        "shortcode": shortcode,
        "child_comment_count": 20,
        "fetch_comment_count": 100,
        "parent_comment_count": 24,
        "has_threaded_comments": True,
    }
    url = "https://www.instagram.com/graphql/query/?query_hash=b3055c01b4b222b8a47dc12b090e4e64&variables="
    result = httpx.get(
        url=url + quote(json.dumps(variables)),
        headers={"x-ig-app-id": INSTAGRAM_APP_ID},
    )
    print(url + quote(json.dumps(variables)))
    print(result.status_code)
    data = json.loads(result.content)
    return data["data"]["shortcode_media"]

# Example usage:
posts = scrape_post("https://www.instagram.com/p/CuE2WNQs6vH/")
print(json.dumps(posts, indent=2, ensure_ascii=False))


scraping instagram post: CuE2WNQs6vH
https://www.instagram.com/graphql/query/?query_hash=b3055c01b4b222b8a47dc12b090e4e64&variables=%7B%22shortcode%22%3A%20%22CuE2WNQs6vH%22%2C%20%22child_comment_count%22%3A%2020%2C%20%22fetch_comment_count%22%3A%20100%2C%20%22parent_comment_count%22%3A%2024%2C%20%22has_threaded_comments%22%3A%20true%7D
200
{
  "__typename": "GraphVideo",
  "id": "3135870261135649735",
  "shortcode": "CuE2WNQs6vH",
  "dimensions": {
    "height": 1333,
    "width": 750
  },
  "gating_info": null,
  "fact_check_overall_rating": null,
  "fact_check_information": null,
  "sensitivity_friction_info": null,
  "sharing_friction_info": {
    "should_have_sharing_friction": false,
    "bloks_app_url": null
  },
  "media_overlay_info": null,
  "media_preview": "ABgqqvHHEMj8TmqEkgPSrVymfx5qu8DIpbAxwM59aleZbXSxVHWirCW7Fd5/AetFWtQUTXFsmDwfu8cnrg5IBOT26celSxwoVwyknIOOemB+fJP/AOqucyTzmlDEdz+dRYSdjpLiKNh0xtyRyeeCBj8SDkdcelFc0WJoppWDmfcmkTqyjABwfrUFacYBEoPIwT+tZdMkQ+tFKaKAP//Z",
  "di

In [40]:
posts.keys()

dict_keys(['__typename', 'id', 'shortcode', 'dimensions', 'gating_info', 'fact_check_overall_rating', 'fact_check_information', 'sensitivity_friction_info', 'sharing_friction_info', 'media_overlay_info', 'media_preview', 'display_url', 'display_resources', 'accessibility_caption', 'dash_info', 'has_audio', 'video_url', 'video_view_count', 'video_play_count', 'is_video', 'tracking_token', 'upcoming_event', 'edge_media_to_tagged_user', 'edge_media_to_caption', 'can_see_insights_as_brand', 'caption_is_edited', 'has_ranked_comments', 'like_and_view_counts_disabled', 'edge_media_to_parent_comment', 'edge_media_to_hoisted_comment', 'edge_media_preview_comment', 'comments_disabled', 'commenting_disabled_for_viewer', 'taken_at_timestamp', 'edge_media_preview_like', 'edge_media_to_sponsor_user', 'is_affiliate', 'is_paid_partnership', 'location', 'nft_asset_info', 'viewer_has_liked', 'viewer_has_saved', 'viewer_has_saved_to_collection', 'viewer_in_photo_of_you', 'viewer_can_reshare', 'owner', 'i